# DataFrame MultiIndex and Index Hierarchy 

In [3]:
import numpy as np
import pandas as pd

In [5]:
outside = ['G1','G1','G1', 'G2', 'G2','G2']
inside = [1,2,3,1,2,3]
higher_index = list(zip(outside,inside))
higher_index = pd.MultiIndex.from_tuples(higher_index)
higher_index

MultiIndex(levels=[['G1', 'G2'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]])

In [6]:
df = pd.DataFrame( np.random.randn(6,2) , higher_index, ['A','B'] )
df

A         B
G1 1  0.310760 -0.080650
   2  1.250832  0.379854
   3 -0.410324 -0.581752
G2 1  0.565633  0.376744
   2 -1.100120  0.160748
   3  0.702458 -1.565979

In [15]:
df.loc['G1']

,A,B
1,2.641866,0.018362
2,0.941670,-0.584640
3,1.273788,0.300589


In [19]:
df.loc['G1'].loc[1]

A    2.641866
B    0.018362
Name: 1, dtype: float64

In [21]:
df.loc['G2'].loc[2]['B']

0.7939974435980256

In [7]:
df.index.names = ["Groups", "num"]
df

A         B
Groups num                    
G1     1    0.310760 -0.080650
       2    1.250832  0.379854
       3   -0.410324 -0.581752
G2     1    0.565633  0.376744
       2   -1.100120  0.160748
       3    0.702458 -1.565979

xs specifies cross section to get data of any row inside Group G1 or G2. It is difficult to perform that with loc, so we use xs. Following code takes "1" row of level "num" in different groups.

In [8]:
df.xs(1,level ="num")

,A,B
Groups,,
G1,0.310760,-0.080650
G2,0.565633,0.376744


In [9]:
df.xs(3,level="num")

,A,B
Groups,,
G1,-0.410324,-0.581752
G2,0.702458,-1.565979


# Dropping and filling Nan values

In [21]:
d = { 'A' : [3,2,np.nan], 'B': [2,np.nan,9], 'C': [6,4,5] }

In [22]:
df1 = pd.DataFrame(d)

In [23]:
df1

,A,B,C
0,3.0,2.0,6
1,2.0,NaN,4
2,NaN,9.0,5


To drop the rows which have NaN values we use dropna().
To drop the column which has NaN values we do dropna(axis=1).
We can also mention a threshold, if number of NaN values > threshold, then drop the row. 

In [24]:
df1.dropna()

,A,B,C
0,3.0,2.0,6


In [25]:
df1.dropna(axis=1)

,C
0,6
1,4
2,5


In [34]:
df['B'][2] = np.nan

In [35]:
df

,A,B,C
0,3.0,NaN,6.0
1,2.0,2.0,NaN
2,NaN,NaN,5.0


In [36]:
df.dropna(thresh=2)

,A,B,C
0,3.0,NaN,6.0
1,2.0,2.0,NaN


In [37]:
df.fillna(value="Fill")

,A,B,C
0,3,Fill,6
1,2,2,Fill
2,Fill,Fill,5


In [39]:
df['A'].fillna(value=df['A'].mean())

0    3.0
1    2.0
2    2.5
Name: A, dtype: float64

# DataFrame GroupBy

In [40]:
data = {'Company' : ['Google', 'Google', 'Microsoft','Microsoft', 'FB', 'FB'], 'Person' : ['Sam', 'Van', 'Aki', 'Kakku', 'Tom', 'Nik'],'Sales' : [200, 230, 120,400,300,350] }

In [43]:
df2 = pd.DataFrame(data)
df2

,Company,Person,Sales
0,Google,Sam,200
1,Google,Van,230
2,Microsoft,Aki,120
3,Microsoft,Kakku,400
4,FB,Tom,300
5,FB,Nik,350


In [48]:
grouped = df2.groupby('Company')

In [49]:
grouped.sum()

,Sales
Company,
FB,650
Google,430
Microsoft,520


In [50]:
grouped.mean()

,Sales
Company,
FB,325
Google,215
Microsoft,260


groupBy('Company') will try to sum/mean/max/min etc each column. But such operations are invalid on column containing strings. So, it returns only 'Sales'. If the operation is applicable to n columns, the result contains n columns.Also, to access a particular column of the grouped result, we can use indexing

In [53]:
grouped.mean().loc['Google']

Sales    215
Name: Google, dtype: int64

In [54]:
grouped.describe()

Sales                                                      
          count   mean         std    min    25%    50%    75%    max
Company                                                              
FB          2.0  325.0   35.355339  300.0  312.5  325.0  337.5  350.0
Google      2.0  215.0   21.213203  200.0  207.5  215.0  222.5  230.0
Microsoft   2.0  260.0  197.989899  120.0  190.0  260.0  330.0  400.0

In [55]:
grouped.describe().transpose()

Company              FB      Google   Microsoft
Sales count    2.000000    2.000000    2.000000
      mean   325.000000  215.000000  260.000000
      std     35.355339   21.213203  197.989899
      min    300.000000  200.000000  120.000000
      25%    312.500000  207.500000  190.000000
      50%    325.000000  215.000000  260.000000
      75%    337.500000  222.500000  330.000000
      max    350.000000  230.000000  400.000000

In [57]:
grouped.describe().transpose()['FB']

Sales  count      2.000000
       mean     325.000000
       std       35.355339
       min      300.000000
       25%      312.500000
       50%      325.000000
       75%      337.500000
       max      350.000000
Name: FB, dtype: float64